In [1]:
import warnings
import os
warnings.simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"
import logging
logging.basicConfig(force=True)

In [3]:
n_samples = 1000
n_nodes = 20
expected_indegree = 6
seed = 3

# simulate linear Gaussian data
from castle.datasets import IIDSimulation, DAG
W = DAG.erdos_renyi(n_nodes=n_nodes, n_edges=n_nodes * expected_indegree, weight_range=[.5, 2], seed=seed)
dataset = IIDSimulation(W=W, n=n_samples)
groundtruth_dag, data = dataset.B, dataset.X

# run ALIAS
from alias.ALIAS import ALIAS
est = ALIAS(
    X=data,
    n_envs=64,                  # number of environments, similar to batch_size in RL_BIC & CORL
    steps_per_env=20000,        # total steps is steps_per_env x n_envs
    dag_scorer_cls='BIC',       # using BIC score (Bayesian Information Criterion), custom scoring methods can be defined in alias.DAGScore
    dag_scorer_kwargs={'regressor': 'LinearRegression', 'variant': 'EV'}, # BIC-EV with Linear Regression, custom regressors can be defined in alias.DAGScore
)

# pruning the returned DAG may be beneficial
from utils.dag_pruners import prune_by_linear_weight
pruned_dag = prune_by_linear_weight(X=data, dag=est, threshold=.3)

# evaluate
from castle.metrics import MetricsDAG
print(MetricsDAG._count_accuracy(pruned_dag, groundtruth_dag))

{'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 112, 'precision': np.float64(1.0), 'recall': np.float64(1.0), 'F1': np.float64(1.0), 'gscore': np.float64(1.0)}
